### First pass of exploring FNAL data

In [1]:
import sys, os
import pandas as pd

In [2]:
DATA_DIR = "../data"

In [3]:
lib_path = '../'
if lib_path not in sys.path:
    sys.path.append(lib_path)

In [4]:
#!pip install fastparquet

In [5]:
fname_in = f"{DATA_DIR}/fifebatch-history-v3-2024.01.01-0002.parquet"
fname_out = f"{DATA_DIR}/anonymized-batch-0002.parquet"

In [6]:
fname_hmac_out = f"{DATA_DIR}/hmac-anonymized-batch-0002.parquet"
fname_fpe_out = f"{DATA_DIR}/fpe-anonymized-batch-0002.parquet"

In [7]:
df = pd.read_parquet(fname_in, engine="fastparquet")  # or engine="fastparquet"
print(df.dtypes)       # Column names + types

@timestamp                datetime64[us]
@version                          object
AccountingGroup                   object
AccountingGroupOSG                object
AcctGroup                         object
                               ...      
x509UserProxyFirstFQAN            object
x509UserProxyVOName               object
x509userproxy                     object
x509userproxysubject              object
xcount                            object
Length: 595, dtype: object


In [8]:
df.head()

,@timestamp,@version,AccountingGroup,AccountingGroupOSG,AcctGroup,AcctGroupUser,AllowOpportunistic,Args,Arguments,AutoClusterAttrs,...,use_x509userproxy,x509UserProxyEmail,x509UserProxyExpiration,x509UserProxyExpiration_ms,x509UserProxyFQAN,x509UserProxyFirstFQAN,x509UserProxyVOName,x509userproxy,x509userproxysubject,xcount
0,NaT,1,group_gm2.prod.gm2pro,None,None,None,None,multipleroot True muon True noGrid False input...,None,None,...,None,None,2024-01-07 16:58:02,1.704668e+12,None,None,None,gm2pro.Production.proxy,/DC=org/DC=incommon/C=US/ST=Illinois/O=Fermi R...,None
1,NaT,1,group_icarus.pro.icaruspro,None,None,None,None,--debug --find_setups --source-unquote /cvmfs/...,None,None,...,None,None,2024-01-07 16:58:07,1.704668e+12,None,None,None,icaruspro.Production.proxy,/DC=org/DC=incommon/C=US/ST=Illinois/O=Fermi R...,None
2,NaT,1,group_icarus.pro.icaruspro,None,None,None,None,--debug --find_setups --source-unquote /cvmfs/...,None,None,...,None,None,2024-01-07 16:58:07,1.704668e+12,None,None,None,icaruspro.Production.proxy,/DC=org/DC=incommon/C=US/ST=Illinois/O=Fermi R...,None
3,NaT,1,group_uboone.prod.uboonepro,None,None,None,None,--nfile 1 --group uboone -g -c wrapper.fcl --u...,None,None,...,None,None,2024-01-07 16:58:08,1.704668e+12,None,None,None,uboonepro.Production.proxy,/DC=org/DC=incommon/C=US/ST=Illinois/O=Fermi R...,None
4,NaT,1,group_pip2.mars.amakovec,None,None,None,None,None,None,None,...,None,amakovec@fnal.gov,2024-01-07 15:29:15,1.704663e+12,None,None,None,x509up_pip2_MARS_59791,/DC=org/DC=cilogon/C=US/O=Fermi National Accel...,None


In [9]:
from hashlib import sha256
from lib.anonymize_kv_mvp import anonymize_df
from lib.utils_secrets import gen_secret_hex

In [10]:
#key_hex = sha256(b"kms-secret").hexdigest()
key_hex = gen_secret_hex(mode="dev", data_dir="../data")
df_hmac = anonymize_df(
    df,
    columns=["AccountingGroup","x509UserProxyEmail","x509userproxy","x509userproxysubject"],
    key_hex=key_hex,
    mode="hmac",
    email_preserve_domain=True,   # token@domain
)

In [11]:
df_hmac.head(2)

,@timestamp,@version,AccountingGroup,AccountingGroupOSG,AcctGroup,AcctGroupUser,AllowOpportunistic,Args,Arguments,AutoClusterAttrs,...,use_x509userproxy,x509UserProxyEmail,x509UserProxyExpiration,x509UserProxyExpiration_ms,x509UserProxyFQAN,x509UserProxyFirstFQAN,x509UserProxyVOName,x509userproxy,x509userproxysubject,xcount
0,NaT,1,vpnpukz5fin54v5yeb45cf,None,None,None,None,multipleroot True muon True noGrid False input...,None,None,...,None,None,2024-01-07 16:58:02,1.704668e+12,None,None,None,lbuh7uw4tttqgjlqe3dnp5,admdemnkv33g5y7sq57beo,None
1,NaT,1,zlu2bll5t2fsn6aqmpcspe,None,None,None,None,--debug --find_setups --source-unquote /cvmfs/...,None,None,...,None,None,2024-01-07 16:58:07,1.704668e+12,None,None,None,nok2bvvc4atg2xpgaloqyp,rktnuakrlitifmbbh5ouby,None


In [12]:
df_hmac.to_parquet(fname_hmac_out, index=False, engine="fastparquet")

In [13]:
df_fpe = anonymize_df(
    df,
    columns=["AccountingGroup","x509UserProxyEmail","x509userproxy","x509userproxysubject"],
    key_hex=key_hex,
    mode="fpe",
    email_preserve_domain=True,   # only local-part anonymized
)

In [14]:
df_fpe.to_parquet(fname_fpe_out, index=False, engine="fastparquet")